In [1]:
import polars as pl
import numpy as np
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import KFold
from imblearn.under_sampling import RandomUnderSampler

In [2]:
df_polars_raiz = pl.read_parquet('../dataset.parquet')

In [3]:
df_polars = df_polars_raiz.sample(fraction=0.01, seed=42)

In [4]:
import ipaddress

def ip_to_int(ip: str) -> int:
    try:
        return int(ipaddress.ip_address(ip))  # Funciona tanto para IPv4 quanto IPv6
    except ValueError:
        return None

In [5]:
#df_polars = df_polars.with_columns([
#    pl.col('id.resp_h').map_elements(ip_to_int).alias('id.resp_h'),
#    pl.col('id.orig_h').map_elements(ip_to_int).alias('id.orig_h')
#])

In [6]:
df_polars = df_polars.with_columns([
    pl.col('duration').fill_null(0),
    pl.col('orig_bytes').fill_null(0),
    pl.col('resp_bytes').fill_null(0)
])

In [7]:
lista_colunas = df_polars.columns
colunas_para_spearman = ['id.resp_p', 'history', 'conn_state', 'id.orig_p', 'orig_ip_bytes', 'label']             
#['detailed-label', 'id.resp_p', 'history', 'id.orig_h', 'conn_state', 'id.orig_p', 'orig_ip_bytes']
colunas_para_dropar = [col for col in lista_colunas if col not in colunas_para_spearman]
df_polars = df_polars.drop(colunas_para_dropar)

In [8]:
df_polars = df_polars.drop_nulls()

In [9]:
X = df_polars.drop('label')
y = df_polars['label']       

In [10]:
#X = X.to_numpy()
#X[:, 0] = np.array([ip_to_int(ip) for ip in X[:, 0]])

# Treinamento

In [11]:
results = []

In [ ]:
def startTrain(X, y, criterion, bootstrap, max_depth, min_samples_split, n_estimators):
    kfold = KFold(n_splits=5, shuffle=True)
    results_fold = []

    for train_idx, test_idx in kfold.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        #rus = RandomUnderSampler()
        #X_train_resampled, y_train_resampled = rus.fit_resample(X_train_scaled, y_train)

        rf = RandomForestClassifier(
            criterion=criterion, bootstrap=bootstrap, max_depth=max_depth,
            min_samples_split=min_samples_split, n_estimators=n_estimators
        )

        start_training = time.time()
        rf.fit(X_train_scaled, y_train)
        end_training = time.time()

        y_pred = rf.predict(X_test_scaled)
        training_duration = end_training - start_training
        evaluation_duration = time.time() - end_training

        confusion = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = confusion.ravel()
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        specificity = tn / (tn + fp)
        f1 = f1_score(y_test, y_pred)
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
        

        avaliacao = [accuracy, balanced_accuracy, precision, recall, specificity, f1, false_alarm_rate, tn, fp, fn, tp, training_duration, evaluation_duration]
        #print(avaliacao)
        results_fold.append(avaliacao)

    results_fold_array = np.array(results_fold, dtype=np.float32)
    mean_results = np.mean(results_fold_array, axis=0)
    results.append(["RF"] + mean_results.tolist())

In [13]:
#Melhores Hiperparâmetros encontrados: {'criterion': 'entropy', 'bootstrap': False, 'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 200}

In [14]:
for i in range(1,11):
    startTrain(X, y, criterion='entropy',bootstrap=False,max_depth=10, min_samples_split=2, n_estimators=200)
    print(i)

1
2
3
4
5
6
7
8
9
10


In [15]:
metrics_df = pl.DataFrame(
    results,
    schema=['Algorithm', 'Accuracy', 'Balanced Accuracy', 'Precision', 'Recall', 'Specificity', 'F1-score', 'False Alarm Rate', 'tn', 'fp', 'fn', 'tp', 'training_duration', 'evaluation_duration']
)
metrics_df

c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\functools.py:888: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


Algorithm,Accuracy,Balanced Accuracy,Precision,Recall,Specificity,F1-score,False Alarm Rate,tn,fp,fn,tp,training_duration,evaluation_duration
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""rf""",0.995018,0.996747,0.999864,0.994202,0.999293,0.997025,0.000707,17790.599609,12.6,540.599976,92702.796875,47.078415,0.662687
"""rf""",0.995015,0.996727,0.999856,0.994207,0.999247,0.997023,0.000753,17789.800781,13.4,540.200012,92703.203125,45.279976,0.630901
"""rf""",0.995013,0.996744,0.999864,0.994196,0.999292,0.997022,0.000708,17790.599609,12.6,541.200012,92702.203125,47.345024,0.648443
"""rf""",0.995022,0.996767,0.999873,0.994198,0.999337,0.997027,0.000663,17791.400391,11.8,541.0,92702.398438,50.218399,0.68875
"""rf""",0.99502,0.996748,0.999864,0.994204,0.999292,0.997026,0.000708,17790.599609,12.6,540.400024,92703.0,47.676502,0.647617
"""rf""",0.995029,0.996772,0.999873,0.994206,0.999337,0.997032,0.000663,17791.400391,11.8,540.200012,92703.203125,48.293324,0.666763
"""rf""",0.995004,0.996712,0.999851,0.994198,0.999225,0.997017,0.000775,17789.400391,13.8,541.0,92702.398438,44.258251,0.636395
"""rf""",0.995009,0.996719,0.999853,0.994202,0.999235,0.99702,0.000765,17789.599609,13.6,540.599976,92702.796875,40.962429,0.622124
"""rf""",0.995011,0.996724,0.999856,0.994202,0.999247,0.997021,0.000753,17789.800781,13.4,540.599976,92702.796875,43.965023,0.634167


In [16]:
metrics_df.write_csv("metrics_results/unbalanced_rf_metrics_output.csv", separator=';')